# Trouver le chemin le plus cours d'un graph avec l'aide d'un algorithme génétique

Implémentation du graph basé sur https://medium.com/geekculture/how-to-represent-a-graph-data-structure-in-python-9f0df57e33a2

Implémentation de l'algorithme génétique basé sur
https://www.youtube.com/watch?v=nhT56blfRpE

## Voici le graph que j'utilise dans le problème

![image of graph](graph-path-cost.webp)


### Étape 1) Prérequis

In [2]:
from collections import namedtuple
from functools import partial
from random import choices, randrange, randint
from random import random 
import time
from typing import Callable, List, Tuple

global DEBUG
DEBUG=False

#Debug Helper Decorator
def show_details(func):
   def wrapper(*args, **kwargs):
      global DEBUG
      DEBUG = True
      #print(f'{func.__name__} called show_details')
      fResult= func(*args,**kwargs)
      DEBUG = False
      return fResult
   return wrapper 

### Étape 2) La classe graph utilisé dans le problème

In [3]:
from random import randint

class Graph:
    def __init__(self):
        self.graph = dict()
        self.isBidirectional = False

    def addEdge(self, node1, node2, cost):
        if node1 not in self.graph:
            self.graph[node1] = []
        if node2 not in self.graph:
            self.graph[node2] = []

        self.graph[node1].append((node2, int(cost)))

        if self.isBidirectional:
            self.graph[node2].append((node1, int(cost)))  # for undirected graph

    def printGraph(self):
        for source, destination in self.graph.items():
            print(f"{source}-->{destination}")
    
    # Permet de voir si un noeud est connecté a une autre noeud
    def nodeConnected(self, nodeA: str, nodeB: str) -> bool:
        global DEBUG
        
        connectedNodes = list(self.graph.get(nodeA))
        if DEBUG==True:
            print(f'\nEvaluating nodes: {nodeA} and {nodeB}')
            print(f'{nodeA} connected nodes: {connectedNodes}')

        for nodeIndex in range(0, len(connectedNodes)):
            
            if len(connectedNodes) == 0: return False 
            
            if DEBUG==True:print(f'{nodeA} at index {nodeIndex} = {connectedNodes[nodeIndex][0]}')
            
            if connectedNodes[nodeIndex][0] == nodeB:
                if DEBUG == True: print(f'{nodeA} and {nodeB} are connected!')
                return True
        
        return False
    
    def generatePath(self, nodeA: str, nodeB: str, length: int):
        global DEBUG

        start = nodeA
        end = nodeB
        path = ""
        cost = []

        path += start
        pathLength = 1
        if DEBUG==True:
            print('\n==============================\n')
            print('Premier noeud : '+start)

        # Liste de noeuds connecté au premier noeud
        availableNodes = list(self.graph.get(start))
        if DEBUG==True:print('\tNoeuds connecté à '+start+str(availableNodes))

        while True:
            if nodeA == end:
                #print('Noeuds connecté à '+nodeA+str(availableNodes))
                if DEBUG==True:print('Dernier noeud : '+end)
                break        
            elif pathLength > length:
                break
            elif availableNodes is False:
                nodeA = nodeB
                if DEBUG==True:print('Noeud a aucune connection. nouveau noeud choisi :'+nodeA) 
            
            # Store le noeud pour être capable de retourner
            nodeB = nodeA

            # Choisi un des noeuds disponible
            randomIndexNoeud = randint(0, len(availableNodes)-1)
            nodeA = availableNodes[randomIndexNoeud][0]
            cost.append(availableNodes[randomIndexNoeud][1])
            
            path += nodeA
            pathLength += 1 
            
            #print('Noeud choisi : '+nodeA)
            
            # Liste de noeuds connecté au nouveau noeud sélectionné
            availableNodes = list(self.graph.get(nodeA))
            if DEBUG==True:print(f'\tNoeuds connecté à {nodeA}: {availableNodes}')
            
        
        #print('==============================')
        return path, cost
        
        # == Implementation checklist ==
        #   Add nodeA to path X
        #   Check connected nodes X
              # if it has no connected nodes, discard X
              # if nodeB is connected add to path
        #   Randomly select a connected node X
        #   Return path X

### Étape 3) Populer le graph avec les information de graph.txt

In [4]:
g= Graph()
     
while True:
    answer = input("Est-ce que le graph est bi-directionnel? (y/n): ")
    
    match answer:
        case "y":
            g.isBidirectional = True
            break
        case "n":
            break
        case _:
            print("Réponse INVALIDE! svp répondez avec y ou n.")

with open("graph.txt") as f:
    lines = f.readlines()

nodes, edges = lines[0].split()

# first line is number of nodes and edges,
# pair of nodes starts from second line
for i in range(1, len(lines)):
    node1, node2, cost = lines[i].split()
    g.addEdge(node1, node2, int(cost))

### Étape 4) Algorithme Génétique

In [5]:
nodeA = 'A'
nodeB = 'E'

Genome = List[str]
Population = List[Genome]

GenerateFunc = Callable[[int], Genome]
FitnessFunc = Callable[[Genome], int]
PopulateFunc = Callable[[int, GenerateFunc, int], Population]
SelectionFunc = Callable[[Population, FitnessFunc], Tuple[Genome, Genome]]
CrossoverFunc = Callable[[Genome, Genome], Tuple[Genome, Genome]]
MutationFunc = Callable[[Genome], Genome]

global paths
paths = []

def generate_population(size: int, genome_length: int) -> Population:
    global paths
    for _ in range(size):
        paths.append(graph_generate_genome(genome_length)) 
    return paths

def selection_pair(population: Population, fitness_func: FitnessFunc) -> Population:
    return choices(
        population=population,
        weights=[fitness_func(genome) for genome in population],
        k=2 
    )

def graph_generate_genome(length: int) -> Genome:
    global DEBUG
    global paths
    
    if DEBUG == True: DEBUG=False

    newGenome = list(g.generatePath(nodeA, nodeB, length))
    
    if DEBUG == False: DEBUG = True
    
    if DEBUG==True:
        print(f'\nGénome généré:{newGenome}')
        print('Current Population:') if paths else print('Current Population: Empty')
    
        for p in paths:
            print(p[0])
    
    if newGenome not in paths:
        return newGenome

    # Vérifie si le nouveau génome est dans la population
    for _ in range(len(paths)):
        
        if newGenome not in paths:
            break
        else:
            #print(gene)
            print(f'{newGenome[0]} already exists! Generating new genome...')
            #Regenerate a new Genome
            
            if DEBUG == True: DEBUG=False
            
            newGenome = list(g.generatePath(nodeA, nodeB, length))

            if DEBUG == False: DEBUG = True

    if DEBUG==True: print(f'{newGenome[0]} created!\tCost={sum(newGenome[1])}\n')
    
    return newGenome

@show_details
def graph_fitness(genome: Genome) -> int: 
    
    #Genomes with a lower path cost rank higher, hence the 1000/cost
    #print(f'{POPULATION_SIZE*100} / {sum(genome[1])} = {POPULATION_SIZE*100/sum(genome[1])}')
    fitnessVal = int(POPULATION_SIZE*100/sum(genome[1]))
    global DEBUG
    if DEBUG==True: print(f'\n === FITCHECK ===\n{genome[0]}\tCOST={sum(genome[1])} SCORE={fitnessVal}') 

    return fitnessVal

@show_details   
def graph_single_point_crossover(a: Genome, b: Genome) -> Tuple[Genome, Genome]:
    global DEBUG
    
    if DEBUG==True: print('\nCriss-cross time\n'+'Genome A : '+str(a)+'\tGenome B : '+str(b))
    
    # Prend la taille du génome le plus petit pour éviter index out of bound
    length = len(b[0]) if len(a[0]) > len(b[0]) else len(a[0])
    
    if DEBUG==True: print('Genome A length: '+str(len(a[0]))+' Genome B length: '+str(len(b[0])))

    if length < 2:
        return a,b

    p = randint(1, length - 1)
    
    if DEBUG==True: print('entre 0 et '+str(length -1)+' : index généré : '+str(p))

    #Parties gauche et droite du génome A et leur cout
    aLeft = a[0][0:p]
    aLCost = a[1][0:p-1]
    aRight = a[0][p:]
    aRCost = a[1][p:]
    
    #Parties gauche et droite du génome B et leur cout
    bLeft = b[0][0:p]
    bLCost = b[1][0:p-1]
    bRight = b[0][p:]
    bRCost = b[1][p:]

    # Check if genomes are the same
    if a[0] == b[0]:
        if DEBUG==True:print('Same genomes. Skipping..')
        return a,b
    
    # Check if nodes are the same
    if aLeft == bLeft:
        if DEBUG==True:print('Same first half. Skipping..')
        return a,b
    
    if aRight == bRight:
        if DEBUG==True:print('Same second half. Skipping..')
        return a,b

    # Checking genome halves compability
    if g.nodeConnected(aLeft[-1], bRight[0]) == False:
        if DEBUG==True:print(f'Incompatible genome halves ({aLeft[-1]} and {bRight[0]} don\'t connect).  Skipping..')
        return a,b # Returns Genomes as they are not compatible
    
    if g.nodeConnected(bLeft[-1], aRight[0]) == False:
        if DEBUG==True:
            print('Incompatible genome halves. Skipping..')
            print(f'Incompatible genome halves ({bLeft[-1]} and {aRight[0]} don\'t connect).  Skipping..')
        return a,b # Returns Genomes as they are not compatible

    #if we're here it means the nodes are connected and the genomes are compatible
    if DEBUG==True:
        print('Genome A Left : '+aLeft+' Cost='+str(aLCost))
        print('Genome A Right : '+aRight+' Cost='+str(aRCost))
        print('Genome B Left : '+bLeft+' Cost='+str(bLCost))
        print('Genome B Right : '+bRight+' Cost='+str(bRCost))

    #Path for new genomes
    a[0] = aLeft + bRight
    b[0] = bLeft + aRight

    if DEBUG == True: DEBUG = False
    
    #Get cost between Left and right parts
    alrCost = [get_cost(aLeft[-1], bRight[0])]
    blrCost = [get_cost(bLeft[-1], aRight[0])]
    
    DEBUG = True

    #Cost for new genomes
    a[1] = aLCost + alrCost+ bRCost
    b[1] = bLCost + blrCost + aRCost
    
    if DEBUG==True: 
        print('\nThe children have been born!\n'+'Child Genome AB : '+str(a)+'\tGenome BA : '+str(b))
    
    # Return modified genomes
    return a, b

def get_cost(l: str, r: str) -> int:
    v = 0
    d =g.graph.get(l)
    if DEBUG==True:print(l+' data: '+ str(d))

    if DEBUG==True:print(g.graph)
    for i in g.graph:
        if DEBUG==True:print(i)
        if i == l:
            for _ in range(len(d)):
                if DEBUG==True:
                    print('index:'+str(_))
                    print(str(d[_][0])+' v='+str(d[_][1]))
                            
                if d[_][0] == r:
                    v = d[_][1]
                    break
        if v != 0:
            break
    if DEBUG==True:print(v)
    return v
    
@show_details
def graph_mutation(genome: Genome, num: int=1, probability: float = 0.25) -> Genome:
    global DEBUG
    global paths
    
    for _ in range(num):
        index = randrange(len(genome[0]))
        if random() < probability: 
            if DEBUG==True:
                print('\nMUTATION\n')
                print(f'Index:{index}')
                print(f'{genome[0]} Cost: {genome[1]}')
            
            modifiablePath = list (genome[0])
            del modifiablePath[index+1:]
            #Remove cost to chopped path
            del genome[1][index:]
            
            genome[0] = ''.join(modifiablePath)
            if DEBUG==True:print(f'After delete\n{genome[0]} Cost: {genome[1]}')
            
            if DEBUG == True: DEBUG = False

            # add part of new generated genome and new cost
            path, cost = g.generatePath(genome[0][index], nodeB, GENOME_LENGTH-len(modifiablePath))
            
            DEBUG = True

            print(f'Path generated:{path}')
            print(f'Generated Path cost:{cost}')
            
            #Remove duplicated node
            del modifiablePath[-1]
            modifiablePath += path
            genome[0] = ''.join(modifiablePath)

            genome[1]+=cost
            
            paths+=genome

            if DEBUG==True:print('Path: '+str(genome[0])+' Cost: '+str(genome[1]))
    
    return genome

#@show_details
def run_evolution(
    populate_func: PopulateFunc,
    fitness_func: FitnessFunc,
    fitness_limit: int,
    selection_func: SelectionFunc = selection_pair,
    crossover_func: CrossoverFunc = graph_single_point_crossover,
    mutation_func: MutationFunc = graph_mutation,
    generation_limit: int = 100
) -> Tuple[Population, int]:
    global DEBUG
    
    if DEBUG == True : DEBUG == False
    population = populate_func()
    if DEBUG == False: DEBUG=True

    if DEBUG==True:
        print('Gen 0\nUNSORTED')
        for gene in population:
            print(gene) 

    for i in range(generation_limit):
        if DEBUG==True:print(f'\nGen {i+1}')
        
        if DEBUG == True : DEBUG == False
        
        population = sorted(
            population,
            key=lambda genome: fitness_func(genome),
            reverse=True
        )
        
        if DEBUG == False: DEBUG=True

        if DEBUG==True:
            print(f'SORTED')
            for gene in population: 
                print(gene) 

        next_generation = population[0:2]
        if DEBUG==True: print(f'TOP 2 {next_generation}')

        for j in range(int(len(population) / 2) - 1):
            parents = selection_func(population, fitness_func)
            offspring_a, offspring_b = crossover_func(parents[0], parents[1])
            offspring_a = mutation_func(offspring_a)
            offspring_b = mutation_func(offspring_b)
            next_generation += [offspring_a, offspring_b]

        population = next_generation

        if DEBUG==True:
            print('\nAfter new genes introduced')
            for gene in population:
                print(gene)
        
        if fitness_func(population[0]) >= fitness_limit:
            break
    
    if DEBUG == True : DEBUG == False
    population = sorted(
            population,
            key=lambda genome: fitness_func(genome),
            reverse=True
        )
    
    if DEBUG == False: DEBUG = True

    print('\nFinal Population')
    for gene in population:
        print(f'{gene[0]} Cost={sum(gene[1])}')    

    return population, i

start = time.time()

GENOME_LENGTH = 100#len(g.graph)
POPULATION_SIZE = 30
population, generations = run_evolution(
    populate_func=partial(
        generate_population, size=POPULATION_SIZE, genome_length=GENOME_LENGTH
    ),
    fitness_func=graph_fitness,
    fitness_limit=GENOME_LENGTH, 
    generation_limit=int(POPULATION_SIZE/2)
)

end = time.time()

print(f"number of generations: {generations}")
print(f"time: {end-start}s")
print(f"best solution: {population[0]}")


Génome généré:['ABE', [5, 7]]
Current Population: Empty

Génome généré:['ABE', [5, 7]]
Current Population:
ABE
ABE already exists! Generating new genome...
ABE created!	Cost=12


Génome généré:['ABDE', [5, 2, 13]]
Current Population:
ABE
ABE

Génome généré:['ACDE', [9, 3, 13]]
Current Population:
ABE
ABE
ABDE

Génome généré:['ABE', [5, 7]]
Current Population:
ABE
ABE
ABDE
ACDE
ABE already exists! Generating new genome...
ABDE already exists! Generating new genome...
ABE already exists! Generating new genome...
ACDE already exists! Generating new genome...
ABDABE created!	Cost=20


Génome généré:['ABDE', [5, 2, 13]]
Current Population:
ABE
ABE
ABDE
ACDE
ABDABE
ABDE already exists! Generating new genome...
ABE already exists! Generating new genome...
ACDACDACDACDE created!	Cost=64


Génome généré:['ABDE', [5, 2, 13]]
Current Population:
ABE
ABE
ABDE
ACDE
ABDABE
ACDACDACDACDE
ABDE already exists! Generating new genome...
ACDE already exists! Generating new genome...
ACDE already exists! 